# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных
* 		children — количество детей в семье
* 		days_employed — общий трудовой стаж в днях
* 		dob_years — возраст клиента в годах
* 		education — уровень образования клиента
* 		education_id — идентификатор уровня образования
* 		family_status — семейное положение
* 		family_status_id — идентификатор семейного положения
* 		gender — пол клиента
* 		income_type — тип занятости
* 		debt — имел ли задолженность по возврату кредитов
* 		total_income — ежемесячный доход
* 		purpose — цель получения кредита

## Оглавление

### [Общая информация](#info)
### [Предобработка данных](#preprocessing)

- [Обработка пропусков](#isna)
- [Замена типа данных](#astype)
- [Обработка дубликатов](#duplicates)
- [Лемматизация ](#lemma)
- [Категоризация данных](#category)

### [Исследование](#research)

- [Зависимость между наличием детей и возвратом кредита в срок](#children)
- [Зависимость между семейным положением и возвратом кредита в срок](#family_status)
- [Зависимость между уровнем дохода и возвратом кредита в срок](#total_income)
- [Как разные цели кредита влияют на его возврат в срок](#purposes)

### [Общий вывод](#conclusion)



## Шаг 1. Откройте файл с данными и изучите общую информацию
<a id='info'></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('~/datasets/data.csv')
data.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.54,1.38,-1.00,0.00,0.00,1.00,20.00
days_employed,19351.0,63046.50,140827.31,-18388.95,-2747.42,-1203.37,-291.10,401755.40
dob_years,21525.0,43.29,12.57,0.00,33.00,42.00,53.00,75.00
education_id,21525.0,0.82,0.55,0.00,1.00,1.00,1.00,4.00
family_status_id,21525.0,0.97,1.42,0.00,0.00,0.00,1.00,4.00
debt,21525.0,0.08,0.27,0.00,0.00,0.00,0.00,1.00
total_income,19351.0,167422.30,102971.57,20667.26,103053.15,145017.94,203435.07,2265604.03


**Вывод**

В столбцах `days_employed` и `total_income` имеются пропуски, минусовые значения и аномально большие значения. Их количество слишком большое (21525 - 19351)/21525* 100 = 10% мы не можем удалить эти строки. Это может повлиять на результат.

## Шаг 2. Предобработка данных
<a id='preprocessing'></a>

### Обработка пропусков 
<a id='isna'></a>

Посчитаем количество строк с пропусками в обоих столбцах и совпадают ли пропуски по обоим столбцам.

In [3]:
data_filtered = data[data['days_employed'].isna()& data['total_income'].isna()]
len(data_filtered)

2174

Строки пропущены в одних и тех же столбцах. Посмотрим первые 10 строк, чтобы выяснить есть ли закономерность по пропускам.

In [4]:
data_filtered.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Закономерность не очевидна, возможно пропуски связаны с тем, что у клиента нет официального источника дохода? Так как восстановить по другим столбцам невозможно, а удалять нельзя, то будем заполнять медианными значениями.

In [5]:
#Теперь этот цикл не нужен, все видно через describe(), хотела удалить, но решила для наглядности закомментировать
#columns = ['children','days_employed', 'dob_years', 'total_income' ]
#for column in columns:
    #print(f'{column}') 
    #print(f'min:{data[column].min()}')
    #print(f'max:{data[column].max()}')
    #print('------------')

В столбце `children` и `days_employed` есть минусовые и аномально большие значения. 

Отрицательные и аномально большие значения стажа (`days_employed`) вероятно даны в часах, их нужно привести к положительным значениям и поделить на 24.


In [6]:
data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed'] >30000, 'days_employed'] = data.loc[data['days_employed'] >30000, 'days_employed'] / 24

Cо столбцом `children` предположим, что -1 скорее всего "нет детей". Данные в проекте выгружались из нескольких систем и у одной из них, похоже, такой формат записи, соответственно заменяем на 0. В случае со значением 20 - скорее всего, опечатка - на клавиатуре 2 и 0 очень близко, заменяем 20 на 2.

In [7]:
data.loc[data['children'] == -1,'children'] = 0
data.loc[data['children'] == 20,'children'] = 2

С столбце `dob_years` есть 0, в наших расчетах этот столбец ни на что не влияет, но все же, будем считать, что это пропуски и заменим их на медианные значения. Снова проверим

In [8]:
data.loc[data['dob_years'] == 0,'dob_years'] = data['dob_years'].median()
data.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.48,0.76,0.00,0.00,0.00,1.00,5.00
days_employed,19351.0,4641.64,5355.96,24.14,927.01,2194.22,5537.88,18388.95
dob_years,21525.0,43.49,12.22,19.00,34.00,42.00,53.00,75.00
education_id,21525.0,0.82,0.55,0.00,1.00,1.00,1.00,4.00
family_status_id,21525.0,0.97,1.42,0.00,0.00,0.00,1.00,4.00
debt,21525.0,0.08,0.27,0.00,0.00,0.00,0.00,1.00
total_income,19351.0,167422.30,102971.57,20667.26,103053.15,145017.94,203435.07,2265604.03


Теперь мы можем заполнить пропуски в столбце `days_employed` медианными значениями по критерию `children` — так как количество детей является одним из критериев исследования.

In [9]:
for children in data['children'].unique():
    median = data.loc[data['children'] == children, 'days_employed'].median()
    data.loc[(data['days_employed'].isna()) & (data['children'] == children), 'days_employed']= median

len(data[data['days_employed'].isna()])  

0

Сделаем тоже самое для столбца `total_income`

In [10]:
for children in data['children'].unique():
    median = data.loc[data['children'] == children, 'total_income'].median()
    data.loc[(data['total_income'].isna()) & (data['children'] == children), 'total_income']= median

len(data[data['total_income'].isna()])

0

Проверим на корректность столбцы с категориальными данными, для этого выведем уникальные значения.

In [11]:
columns = ['education','education_id','family_status','family_status_id','gender','income_type','debt']

for column in columns:
    print(f'{column}:') 
    print(data[column].unique())
    print("------------")

education:
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
------------
education_id:
[0 1 2 3 4]
------------
family_status:
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
------------
family_status_id:
[0 1 2 3 4]
------------
gender:
['F' 'M' 'XNA']
------------
income_type:
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
------------
debt:
[0 1]
------------


В столбцах `education` и `family_status` есть одинаковые названия в разных регистрах.Приведем их к нижнему регистру. Проверим, что значения в столбцах `education_id` и `family_status_id` соответствуют значениям `education` и `family_status` 

<a id='agg'></a>

In [12]:
data['education'] = data['education'].str.lower()

data_grouped_education = data.groupby(['education','education_id']).agg({'education_id':'count'})
data_grouped_education

,,education_id
education,education_id,
высшее,0,5260
начальное,3,282
неоконченное высшее,2,744
среднее,1,15233
ученая степень,4,6


In [13]:
data['family_status'] = data['family_status'].str.lower()

data_grouped_family = data.groupby(['family_status','family_status_id']).agg({'family_status':'count'})
data_grouped_family

,,family_status
family_status,family_status_id,
в разводе,3,1195
вдовец / вдова,2,960
гражданский брак,1,4177
женат / замужем,0,12380
не женат / не замужем,4,2813


Столбцы `education_id` и `family_status_id` не используются дальше, есть смысл их удалить, так как информация дублируется.

In [14]:
data = data.drop(columns = ['education_id', 'family_status_id'])
data.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,женат / замужем,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,женат / замужем,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,женат / замужем,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,женат / замужем,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,гражданский брак,F,пенсионер,0,158616.077870,сыграть свадьбу


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       21525 non-null  int64  
 1   days_employed  21525 non-null  float64
 2   dob_years      21525 non-null  int64  
 3   education      21525 non-null  object 
 4   family_status  21525 non-null  object 
 5   gender         21525 non-null  object 
 6   income_type    21525 non-null  object 
 7   debt           21525 non-null  int64  
 8   total_income   21525 non-null  float64
 9   purpose        21525 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 1.6+ MB


### Замена типа данных 
<a id='astype'></a>

Данные занимают 1.6 MB, оптимизируем типы данных для разных столбцов.

In [16]:
columns = ['education', 'family_status', 'gender', 'income_type']
for column in columns:
    try:
        data[column] = data[column].astype('category')
    except:
        print(f'Не получается преобразовать {column} в категорию!')

columns = ['children','dob_years', 'debt']
for column in columns:
    try:
        data[column] = data[column].astype('int8')
    except:
        print(f'Не получается преобразовать {column} в int8!')
try:
    data['days_employed'] = data['days_employed'].astype('int16')
    data['total_income'] = data['total_income'].astype('int32')
except:
    print('Не получается преобразовать!')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   children       21525 non-null  int8    
 1   days_employed  21525 non-null  int16   
 2   dob_years      21525 non-null  int8    
 3   education      21525 non-null  category
 4   family_status  21525 non-null  category
 5   gender         21525 non-null  category
 6   income_type    21525 non-null  category
 7   debt           21525 non-null  int8    
 8   total_income   21525 non-null  int32   
 9   purpose        21525 non-null  object  
dtypes: category(4), int16(1), int32(1), int8(3), object(1)
memory usage: 442.5+ KB


**Вывод**


Использовался метод `astype()`, так как эти столбцы предполагают целые числа, а также применен метод `try/except` для выявления потенциальных ошибок.

### Обработка дубликатов  
<a id='duplicates'></a>

Посчитаем количество дубликатов, удалим явные дубликаты и проверим, что их нет

In [17]:
print(f'Количество дубликатов до: {data.duplicated().sum()}')
data = data.drop_duplicates().reset_index(drop=True)
print(f'Количество дубликатов после: {data.duplicated().sum()}')

Количество дубликатов до: 72
Количество дубликатов после: 0


**Вывод**

Для удаления дубликатов использовали метод `drop_duplicates()`, так как в данных нет уникальных id, поэтому мы можем удалить только явные дубликаты. 

Возможные причины: дубликаты могли появиться при объединении данных из нескольких баз.

### Лемматизация 
<a id='lemma'></a>

Лемматизируем столбец `purpose`:

In [18]:
m = Mystem()
text = ' '.join(data['purpose'].unique())
lemmas = m.lemmatize(text)
Counter(lemmas).most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

**Вывод**

При лемматизации списка выявлены самые частые существительные - это будут основные категории:

- недвижимость

- автомобиль

- образование

- свадьба

### Категоризация данных
<a id='category'></a>

Наишем функцию для `apply`, которая будет лематизировать последовательно каждую ячейку столбца `purpose` и проверять, какая из категорий обнаруживается в лемматизированном варианте. Посмотрим первые 5 строк.

In [19]:
def categorize(row):
    purpose = row['purpose']
    lemmas_purpose = m.lemmatize(purpose)
    if ('недвижимость' in lemmas_purpose or 'жилье' in lemmas_purpose or 'строительство'in lemmas_purpose):
        return 'недвижимость'
    elif ('автомобиль' in lemmas_purpose):
        return 'автомобиль'
    elif ('образование' in lemmas_purpose):
        return 'образование'
    elif ('свадьба' in lemmas_purpose):
        return 'свадьба'
    else:
        return 'другое'
data['purposes_category'] = data.apply(categorize, axis=1)
data['purposes_category'] = data['purposes_category'].astype('category')
data.head()   

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,purposes_category
0,1,8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Добавили столбец `purposes_category`, в котором цели объединены в категории.

## Шаг 3. Ответьте на вопросы
<a id='research'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
<a id='children'></a>

In [20]:
def children_category(row):
    if row['children'] == 0:
        return '0'
    elif row['children'] == 1:
        return '1'
    elif row['children'] == 2:
        return '2'
    else:
        return '> 2'
data['children'] = data.apply(children_category, axis =1)

report_children = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_children.columns = ['debt', 'total', '%']
report_children = report_children.sort_values(by = ['%'], ascending = False)
report_children['%'] *=100
report_children['%'] = report_children['%'].round(2)
report_children

,debt,total,%
children,,,
2,202.0,2128,9.49
1,444.0,4808,9.23
> 2,31.0,380,8.16
0,1064.0,14137,7.53


**Вывод**

Использовали свобную таблицу `pivot_table` по столбцу `children`, значения взяты из колонки `debt`, к ней применены функции суммирования, подсчет количества и их отношение.

По количеству детей максимальное количество должников среди клиентов с **2мя детьми (9.49%)**, немного меньше, если **1 ребенок (9.23%)** и минамальна, если детей нет **(7.53%)**.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?
<a id='family_status'></a>

In [21]:
report_family = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_family.columns = ['debt', 'total', '%']
report_family = report_family.sort_values(by = ['%'], ascending = False)
report_family['%'] *=100
report_family['%'] = report_family['%'].round(2)
report_family

,debt,total,%
family_status,,,
не женат / не замужем,274.0,2810,9.75
гражданский брак,388.0,4150,9.35
женат / замужем,931.0,12339,7.55
в разводе,85.0,1195,7.11
вдовец / вдова,63.0,959,6.57


**Вывод**

Использовали сводную таблицу `pivot_table` по столбцу `family_status`, значения взяты из колонки `debt`, к ней применены функции суммирования, подсчет количества и их отношение.

Больше всего должников среди тех, кто **не женат / не замужем (9.75%)** и состоит в **гражданском браке (9.35%)**. Существенный отрыв в сторону уменьшения доли должников, если статус **женат / замужем (7.55%)**

Меньше всего должников в статусе **вдовец / вдова (7.11%)** и в **разводе (6.57%)**.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<a id='total_income'></a>

Напишем функцию для `apply`, которая будет проверять каждую строку столбца `total_income` по уровню дохода и добавит новый столбец. Выведим сводную таблицу, которая будет отображать количество должников, общее количество в каждой категории по уровню дохода и их отношение в %, отсортируем в порядке убывания доли  

In [22]:
min_income = 50000
mid_income = 100000
mid_lar_income = 150000
lar_income = 200000

def income(row):
    if row['total_income'] < min_income:
        return 'менее 50 000'
    elif row['total_income'] < mid_income:
        return '50 000 - 100 000'
    elif row['total_income'] < mid_lar_income:
        return '100 000 - 150 000'
    elif row['total_income'] < lar_income:
        return '150 000 - 200 000'
    else:
        return 'более 200 000'
            
data['income_category'] = data.apply(income, axis =1)

report_income = data.pivot_table(index = ['income_category'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_income.columns = ['debt', 'total', '%']
report_income = report_income.sort_values(by = ['%'], ascending = False)
report_income['%'] *=100
report_income['%'] = report_income['%'].round(2)
report_income  

,debt,total,%
income_category,,,
150 000 - 200 000,374.0,4162,8.99
100 000 - 150 000,655.0,7762,8.44
50 000 - 100 000,331.0,4091,8.09
более 200 000,358.0,5066,7.07
менее 50 000,23.0,372,6.18


**Вывод**

По уровню дохода в группе риска находятся те, у кого уровень дохода в диапазоне **150 000 - 200 000 руб. (8.99%)**, меньше всего те, у кого уровень дохода **менее 50 000 руб. (6.18%)**.

- Как разные цели кредита влияют на его возврат в срок?
<a id='purposes'></a>

In [23]:
report_purpose = data.pivot_table(index = ['purposes_category'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_purpose.columns = ['debt', 'total', '%']
report_purpose = report_purpose.sort_values(by = ['%'], ascending = False)
report_purpose['%'] *=100
report_purpose['%'] = report_purpose['%'].round(2)
report_purpose

,debt,total,%
purposes_category,,,
автомобиль,403.0,4306,9.36
образование,370.0,4013,9.22
свадьба,186.0,2323,8.01
недвижимость,782.0,10811,7.23


**Вывод**

Использовали сводную таблицу `pivot_table` по столбцу `purposes_category`, значения взяты из колонки `debt`, к ней применены функции суммирования, подсчет количества и их отношение.

Больше всего должников среди тех, кто взял кредит на **автомобиль (9.36%)**, меньше всего среди тех, кто брал кредит на **недвижимость (7.23%)**.

## Шаг 4. Общий вывод
<a id='conclusion'></a>

Для построения модели кредитного скоринга нам необходимо было выяснить, влияет ли на факт погашения кредита в срок:

а) семейное положение клиента;

б) количество детей клиента.

Мы использовали статистику о платёжеспособности клиентов. Итоговые данные рассчитаны в %, по каждой группе, как отношение количества тех, у кого имеется задолженность к общему количеству внутри каждой категории. 

**Выводы:**

- Больше всего задолженность по возврату кредитов у тех, кто **не женат / не замужем (9.75%)** и состоит в **гражданском браке (9.35%)**. Существенный отрыв в сторону уменьшения доли должников, если статус брака официальный **(7.55%)**. 
Меньше всего должников в статусе **вдовец / вдова (7.11%)** и **в разводе (6.57%)**.


- По количеству детей максимальное количество должников среди клиентов с **2мя детьми (9.49%)**, немного меньше, если **1 ребенок (9.23%)** и минамальна, если детей нет **(7.53%)**.


- Дополнительно выяснили, что задолженность по возврату кредитов имеется у тех, кто берет кредит на **автомобиль (9.36%)**, меньше всего - **на недвижимость (7.23%)**. 


- По уровню дохода в группе риска находятся те, у кого уровень дохода в диапазоне **150 000 - 200 000 руб. (8.99%)**, меньше всего те, у кого уровень дохода **менее 50 000 руб. (6.18%)**.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.